In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
sns.set_style('darkgrid')
from scipy import stats
from scipy.stats import skew, norm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
# saving and then dropping the ID column from the test and train
train_ID = train.Id
test_ID = test.Id

train.drop('Id', axis = 1, inplace = True)
test.drop('Id', axis = 1, inplace = True)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(train.GrLivArea, train.SalePrice)
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
# removal of outliers
train = train.drop(train[(train['GrLivArea'] > 4000) & (train['SalePrice'] < 300000)].index)

fig, ax = plt.subplots()
ax.scatter(train['GrLivArea'], train['SalePrice'])
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
# Histogram of SalePrice
sns.distplot(train['SalePrice'], fit = norm)

mu, sigma = norm.fit(train['SalePrice'])
print('\n mu: {:.2f} and sigma: {:.2f}'.format(mu, sigma))

plt.legend(['Normal Dist: $\mu$ = {:.2f} and $\sigma$ = {:.2f}'.format(mu, sigma)], loc = 'best')
plt.ylabel('Frequeuncy')
plt.title('SalePrice Distribution')
plt.show()

# Q-Q plot for SalePrice
res = stats.probplot(train['SalePrice'], plot = plt)
plt.show()

In [ ]:
# Taking log transformation of SalePrice
train['SalePrice'] = np.log1p(train['SalePrice'])

mu, sigma = norm.fit(train['SalePrice'])
print('\n mu: {:.2f} and sigma: {:.2f}'.format(mu, sigma))

sns.distplot(train['SalePrice'], fit = norm)
plt.legend(['Normal Dist: $\mu$ = {:.2f} and $\sigma$ = {:.2f}'.format(mu, sigma)], loc = 'best')
plt.ylabel('Frequeuncy')
plt.title('SalePrice Distribution')
plt.show()

stats.probplot(train['SalePrice'], plot = plt)
plt.show()

In [ ]:
# Concatenating train and test dataset

ntrain = train.shape[0]
ntest = test.shape[0]

y_train = train.SalePrice.values
all_data = pd.concat([train, test]).reset_index(drop = True)
all_data.drop(['SalePrice'], axis=1,inplace = True)
print('Shape of all_data: {}'.format(all_data.shape))

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.sort_values(ascending=False)
missing_data = pd.DataFrame({'missing_values': all_data_na})
missing_data[:20]

In [ ]:
f, ax = plt.subplots(figsize=(10, 7))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na.values)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
train_corr = train.corr()
plt.subplots(figsize = (12, 9))
sns.heatmap(train_corr, robust= True, cmap='magma', vmax = 0.9)

In [ ]:
# Imputing missing values
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

In [ ]:
# Filling with median of neighborhood frontage area
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
    
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
    
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

# Filling up with mode value
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])


# Dropping Utilities
all_data = all_data.drop(['Utilities'], axis=1)

all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [ ]:
# Again checking for missing values

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

In [ ]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [ ]:
# Label Encodeing of some variables which may have some information in their order

from sklearn.preprocessing import LabelEncoder

cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    labelEncoder = LabelEncoder()
    all_data[c] = labelEncoder.fit_transform(all_data[c])
    
print('Shape all_data: {}'.format(all_data.shape))

In [ ]:
# Adding another variable as area related features are important for house prices
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [ ]:
# Getting list of skewed variables
numeric_feat = all_data.dtypes[all_data.dtypes != 'object'].index

skew_feats = all_data[numeric_feat].apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame({'Skew_features': skew_feats})
skewness.head(10)

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print('Number of variables for BoxCox transformation: ' + str(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lambd = 0.15
for feature in skewed_features:
    all_data[feature] = boxcox1p(all_data[feature], lambd)

In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
# Getting back train and test data

train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin, clone
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
# Setting up cross validation strategy
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(abs(cross_val_score(model, train.values, y_train, scoring='neg_mean_squared_error', cv = kf)))
    return rmse

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha = 0.0005, random_state=1))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=1))

In [ ]:
KRR = KernelRidge(alpha = 0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score = rmsle_cv(lasso)
print('\n Lasso score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print('\n ElasticNet score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print('\n KRR score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print('\n Gradient Boosting score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print('\n XGBoost score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print('\n Light GBM score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        for model in self.models_:
            model.fit(X, y)
        return self
    
    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions, axis = 1)

In [ ]:
averaged_model = AveragingModels((ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_model)
print('\n Averaged Model score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
averaged_model = AveragingModels((ENet, GBoost, KRR, lasso, model_lgb, model_xgb))

score = rmsle_cv(averaged_model)
print('\n Averaged Model score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_folds, shuffle = True, random_state=156)
        
        out_of_fold_predictions = np.zeros((X.shape[0],len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X[train_index], y[train_index])
                self.base_models_[i].append(instance)
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([np.column_stack([model.predict(X) for model in base_models]).mean(axis = 1) for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)   

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models=(ENet, GBoost, KRR), meta_model=lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models.fit(train.values, y_train)
stacked_trained_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_trained_pred))

In [ ]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_trained_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)

In [ ]:
submission = pd.read_csv('submission.csv')